In [1]:
from dotenv import load_dotenv

from langchain_community.tools import ArxivQueryRun , WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper , WikipediaAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults

from langchain_groq import ChatGroq


from langgraph.graph import StateGraph , START , END
from langgraph.prebuilt import ToolNode , tools_condition

from typing_extensions import TypedDict
from langgraph.graph.message import add_messages , Annotated 
from langchain.messages import HumanMessage , AIMessage , AnyMessage

import os

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [4]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results= 2 , doc_content_chars_max= 500) 
arxiv = ArxivQueryRun(api_wrapper= arxiv_wrapper)

wiki_wrapper = WikipediaAPIWrapper(top_k_results= 1 , doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper= wiki_wrapper)

tavily = TavilySearchResults()

/var/folders/9c/tpzdhn2d0rv1nscqt75gbvgw0000gn/T/ipykernel_35919/3784891661.py:7: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tavily = TavilySearchResults()


In [5]:
# Custom functions

def add(a : int , b: int) -> int :

    """ Function Returns sum of two numbers

    Args:
        a (int): Integer 
        b (int): Integer 

    Returns:
        int: Integer a+b

    """

    return a+b 

def multiply(a: int, b: int) -> int:
    """Function returns the product of two numbers.

    Args:
        a (int): Integer A
        b (int): Integer B

    Returns:
        int: Integer A * B
    """
    return a * b


def divide(a: int, b: int) -> float:
    """Function returns the division of two numbers.

    Args:
        a (int): Integer A
        b (int): Integer B

    Returns:
        float: Result of A / B

    Raises:
        ValueError: If b is zero.
    """
    if b == 0:
        raise ValueError("Division by zero is not allowed.")
    return a / b



In [6]:
tools = [arxiv , wiki , tavily , add, multiply, divide]

In [7]:
class State(TypedDict) :
    messages : Annotated[list[AnyMessage],add_messages]

In [8]:
graph = StateGraph(State)

In [9]:
llm = ChatGroq(model= "openai/gpt-oss-20b")

In [10]:
llm_with_tools = llm.bind_tools(tools)

In [11]:
# Node Definition

def chatbot(state : State) :

    return {"messages" : [llm_with_tools.invoke(state["messages"])]}

In [12]:
# Add nodes
graph.add_node("chatbot" , chatbot)
graph.add_node("tools" , ToolNode(tools))

graph.add_edge(START , "chatbot")
graph.add_conditional_edges("chatbot" , tools_condition)

graph.add_edge("tools" , "chatbot")


graph_builder = graph.compile()

In [15]:
response= graph_builder.invoke({"messages" : ["What is 2025 + 2025 , Recent AI news"]})

# Agent with memory

In [21]:
from langgraph.checkpoint.memory import MemorySaver

In [22]:
memory = MemorySaver()

In [19]:
graph = StateGraph(State)

In [20]:
# Add nodes
graph.add_node("chatbot" , chatbot)
graph.add_node("tools" , ToolNode(tools))

graph.add_edge(START , "chatbot")
graph.add_conditional_edges("chatbot" , tools_condition)

graph.add_edge("tools" , "chatbot")

In [23]:
graph_memory = graph.compile(checkpointer= memory)

In [24]:
# Specify thread 

config = {"configurable" : {"thread_id" : 1}}

In [25]:
graph_memory.invoke({"messages" : ["What is 25 + 25"]}, config= config)

{'messages': [HumanMessage(content='What is 25 + 25', additional_kwargs={}, response_metadata={}, id='66acb729-6628-4a9e-aa7f-e5a174777b0c'),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to answer simple math. The user asked "What is 25 + 25". We can compute 50. We can also use the add function. Let\'s use the add function.', 'tool_calls': [{'id': 'fc_9c4394fa-8faf-4459-a7b2-90a989312ce9', 'function': {'arguments': '{"a":25,"b":25}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 385, 'total_tokens': 451, 'completion_time': 0.06532435, 'prompt_time': 0.020294459, 'queue_time': 0.008106955, 'total_time': 0.085618809, 'completion_tokens_details': {'reasoning_tokens': 40}}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_3417a662f8', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--6a77518e-065c-41cf-9886-0d2af3a

In [26]:
graph_memory.invoke({"messages" : ["What is that plus 25"]}, config= config)

{'messages': [HumanMessage(content='What is 25 + 25', additional_kwargs={}, response_metadata={}, id='66acb729-6628-4a9e-aa7f-e5a174777b0c'),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to answer simple math. The user asked "What is 25 + 25". We can compute 50. We can also use the add function. Let\'s use the add function.', 'tool_calls': [{'id': 'fc_9c4394fa-8faf-4459-a7b2-90a989312ce9', 'function': {'arguments': '{"a":25,"b":25}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 385, 'total_tokens': 451, 'completion_time': 0.06532435, 'prompt_time': 0.020294459, 'queue_time': 0.008106955, 'total_time': 0.085618809, 'completion_tokens_details': {'reasoning_tokens': 40}}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_3417a662f8', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--6a77518e-065c-41cf-9886-0d2af3a